In [11]:
class node:
    def __init__(self, word, word_count=0, parent=None, link=None):
        # Initialisation d'un nœud dans l'arbre
        self.word = word  # Le mot représenté par ce nœud
        self.word_count = word_count  # Le nombre d'occurrences de ce mot
        self.parent = parent  # Référence au nœud parent (le nœud précédent dans l'arbre)
        self.link = link  # Lien vers un autre nœud du même mot (chaînage entre nœuds du même mot)
        self.children = {}  # Dictionnaire des enfants du nœud, où la clé est le mot et la valeur est le nœud enfant

    def visittree(self):
        # Méthode pour parcourir l'arbre et afficher ses nœuds
        output = [f"{self.word} ({self.word_count})"]  # Ajout du mot et de sa fréquence dans la sortie
        if len(self.children) > 0:  # Si le nœud a des enfants
            for i in self.children.keys():
                output.extend(self.children[i].visittree())  # Appel récursif pour visiter les enfants
        return output


class fptree:
    def __init__(self, data, minsup=400):
        # Initialisation de l'arbre FP avec les données transactionnelles et le seuil de support minimal
        self.data = data  # Données transactionnelles sous forme de liste de transactions
        self.minsup = minsup  # Seuil minimal de support

        # Racine de l'arbre FP (nœud "Null" qui n'a pas de mot associé)
        self.root = node(word="Null", word_count=1)

        # Structures de données pour gérer l'arbre FP et le processus de construction
        self.wordlinesort = []  # Liste des transactions triées par fréquence des mots
        self.nodetable = []  # Table des nœuds pour gérer les liens entre nœuds du même mot
        self.wordsortdic = []  # Liste des mots triés par fréquence
        self.worddic = {}  # Dictionnaire des mots avec leurs comptes d'occurrences
        self.wordorderdic = {}  # Dictionnaire des positions des mots dans le tri

        # Construction de l'arbre FP
        self.construct(data)

    def construct(self, data):
        # Étape 1 : Comptage des occurrences des mots dans les transactions
        for tran in data:  # Parcours des transactions
            for words in tran:  # Parcours des mots dans chaque transaction
                if words in self.worddic.keys():
                    self.worddic[words] += 1  # Incrémentation du compte pour le mot
                else:
                    self.worddic[words] = 1  # Initialisation du compte pour ce mot

        # Suppression des mots dont le support est inférieur au seuil de support minimal
        wordlist = list(self.worddic.keys())
        for word in wordlist:
            if self.worddic[word] < self.minsup:
                del self.worddic[word]  # Suppression du mot du dictionnaire s'il ne satisfait pas le min_sup

        # Tri des mots restants par fréquence décroissante
        self.wordsortdic = sorted(self.worddic.items(), key=lambda x: (-x[1], x[0]))

        # Création de la table des nœuds (pour gérer les liens entre nœuds du même mot)
        t = 0
        for i in self.wordsortdic:
            word = i[0]
            wordc = i[1]
            self.wordorderdic[word] = t  # Enregistrement de la position du mot dans le tri
            t += 1
            wordinfo = {'wordn': word, 'wordcc': wordc, 'linknode': None}
            self.nodetable.append(wordinfo)

        # Étape 2 : Construction de l'arbre FP à partir des transactions filtrées et triées
        for line in data:
            supword = []  # Liste des mots ayant un support suffisant
            for word in line:
                if word in self.worddic.keys():
                    supword.append(word)  # Ajout des mots ayant un support suffisant

            if len(supword) > 0:
                # Trie des mots dans la ligne selon l'ordre des fréquences
                sortsupword = sorted(supword, key=lambda k: self.wordorderdic[k])
                self.wordlinesort.append(sortsupword)
                R = self.root  # Commence à la racine de l'arbre
                for i in sortsupword:
                    if i in R.children.keys():
                        R.children[i].word_count += 1  # Incrémente si le nœud existe déjà
                        R = R.children[i]  # Avance vers le nœud enfant
                    else:
                        # Crée un nouveau nœud pour l'item et l'ajoute à l'arbre
                        R.children[i] = node(word=i, word_count=1, parent=R, link=None)
                        R = R.children[i]
                        # Mise à jour des liens dans la table des nœuds
                        for wordinfo in self.nodetable:
                            if wordinfo["wordn"] == R.word:
                                if wordinfo["linknode"] is None:
                                    wordinfo["linknode"] = R
                                else:
                                    iter_node = wordinfo["linknode"]
                                    while iter_node.link is not None:
                                        iter_node = iter_node.link
                                    iter_node.link = R

    def condtreetran(self, N):
        # Création des transactions pour un arbre conditionnel à partir d'un nœud donné
        if N.parent is None:
            return None

        condtreeline = []
        while N is not None:
            line = []
            PN = N.parent
            while PN.parent is not None:
                line.append(PN.word)  # Ajout des mots depuis le nœud jusqu'à la racine
                PN = PN.parent
            line = line[::-1]  # Inverse l'ordre des mots pour conserver la hiérarchie
            for _ in range(N.word_count):
                condtreeline.append(line)
            N = N.link  # Passe au prochain nœud du même mot
        return condtreeline

    def findfqt(self, parentnode=None):
        # Recherche des ensembles fréquents à partir de l'arbre FP
        if len(list(self.root.children.keys())) == 0:
            return None
        result = []
        sup = self.minsup
        revtable = self.nodetable[::-1]  # Parcours en sens inverse de la table des nœuds
        for n in revtable:
            fqset = [set(), 0]
            if parentnode is None:
                fqset[0] = {n['wordn']}
            else:
                fqset[0] = {n['wordn']}.union(parentnode[0])
            fqset[1] = n['wordcc']  # Fréquence de l'élément
            result.append(fqset)
            condtran = self.condtreetran(n['linknode'])  # Arbre conditionnel pour ce nœud
            contree = fptree(condtran, sup)  # Crée l'arbre FP pour les transactions conditionnelles
            conwords = contree.findfqt(fqset)  # Recherche d'ensembles fréquents dans cet arbre
            if conwords is not None:
                for words in conwords:
                    result.append(words)  # Ajoute les résultats trouvés
        return result

    def checkheight(self):
        # Vérifie si l'arbre FP a une hauteur > 1
        return len(list(self.root.children.keys())) > 0


# Exemple de test
min_sup = 4  # Seuil minimal de support

test_data = [['I1', 'I2', 'I5'],
             ['I2', 'I4'],
             ['I2', 'I3'],
             ['I1', 'I2', 'I4'],
             ['I1', 'I3'],
             ['I2', 'I3'],
             ['I1', 'I3'],
             ['I1', 'I2', 'I3', 'I5'],
             ['I1', 'I2', 'I3']]  # Données de test représentant des transactions

vocabdic = {"I1": "Item1", "I2": "Item2", "I3": "Item3", "I4": "Item4", "I5": "Item5"}  # Dictionnaire de vocabulaire pour afficher les items

fp_tree = fptree(test_data, min_sup)  # Création de l'arbre FP avec les données de test et le seuil minimal de support

print("\n========== Ensembles fréquents ==========")
frequentwordset = fp_tree.findfqt()  # Recherche des ensembles fréquents
frequentwordset = sorted(frequentwordset, key=lambda k: -k[1])  # Tri des ensembles fréquents par fréquence décroissante

# Affichage des ensembles fréquents avec leurs comptes
for word in frequentwordset:
    count = str(word[1]) + "\t"
    words = ' '.join(vocabdic[val] for val in word[0])  # Traduction des indices en mots avec vocabdic
    print(count)



========== Ensembles fréquents ==========
7	
6	
6	
4	
4	
4	
